<a href="https://colab.research.google.com/github/Cerlog/RUC-Bachelor-Project/blob/main/Our_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
# Credit goes to https://sundnes.github.io/solving_odes_in_python/
import numpy as np

class ODESolver:
    def __init__(self, f):
        # Wrap user's f in a new function that always
        # converts list/tuple to array (or let array be array)
        self.f = lambda u, t: np.asarray(f(u, t), float)

    def set_initial_condition(self, U0):
        if isinstance(U0, (float,int)):  # scalar ODE
            self.neq = 1                 # no of equations
            U0 = float(U0)
        else:                            # system of ODEs
            U0 = np.asarray(U0)
            self.neq = U0.size           # no of equations
        self.U0 = U0

    def solve(self, time_points):
        self.t = np.asarray(time_points)
        N = len(self.t)
        if self.neq == 1:  # scalar ODEs
            self.u = np.zeros(N)
        else:              # systems of ODEs
            self.u = np.zeros((N,self.neq))

        # Assume that self.t[0] corresponds to self.U0
        self.u[0] = self.U0

        # Time loop
        for n in range(N-1):
            self.n = n
            self.u[n+1] = self.advance()
        return self.u, self.t

class ForwardEuler(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t

        dt = t[n+1] - t[n]
        unew = u[n] + dt*f(u[n], t[n])
        return unew

class ExplicitMidpoint(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = t[n+1] - t[n]
        dt2 = dt/2.0
        k1 = f(u[n], t[n])
        k2 = f(u[n] + dt2*k1, t[n] + dt2)
        unew = u[n] + dt*k2
        return unew

class RungeKutta4(ODESolver):
    def advance(self):
        u, f, n, t = self.u, self.f, self.n, self.t
        dt = t[n+1] - t[n]
        dt2 = dt/2.0
        k1 = f(u[n], t[n])
        k2 = f(u[n] + dt2*k1, t[n] + dt2)
        k3 = f(u[n] + dt2*k2, t[n] + dt2)
        k4 = f(u[n] + dt*k3, t[n] + dt)
        unew = u[n] + (dt/6.0)*(k1 + 2*k2 + 2*k3 + k4)
        return unew


In [16]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
mpl.rcParams.update(mpl.rcParamsDefault)
#plt.rcParams.update({'text.usetex': True}) # import to make plots nicer
#%config InlineBackend.figure_format = 'svg' # import to make plots nicer
#from ODE_SOLVER import RungeKutta4 # for this import to work - make sure the file ODE_SOLVER is in the same file as your jupyter notebook
from scipy.integrate import solve_ivp
from matplotlib.pyplot import figure
# Set global font size for title, x-label, and y-label
plt.rcParams['axes.titlesize'] = 20
plt.rcParams['axes.labelsize'] = 16

# Set global font size for x and y tick labels
plt.rcParams['xtick.labelsize'] = 16
plt.rcParams['ytick.labelsize'] = 16

# Set global font size for the legend
plt.rcParams['legend.fontsize'] = 16

plt.rcParams['figure.dpi'] = 300


figure(figsize=(15, 10), dpi=300) # set the size of the figure
from ipywidgets import interactive



In [17]:
def show_sol(k1 = 1., k2 = 1., r3 = 1., ET = 0.5, r1=1, UT=0.1, r2=0.075, J3 = 0.3, K2 = 0.3, k4 = 0.2, J4 = 0.5):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        Y = x[0]
        VP = G(r3 * Y, k4, J3, J4) # equation (3)
        UP = x[1]
        dYdT = k1*VP -k2*UP*Y
        dUPdT = r1*ET*((UT-UP)/(k1+(UT-UP))-r2*Y*(UP/(K2+UP)))
        return [dYdT, dUPdT]



    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(t, R, label='Y')
    plt.plot(t, X, label='UP')
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w = interactive(show_sol, k1=(0.1, 10, 0.01), K1=(0.1, 10, 0.01),r3=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), r1=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), r2=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), ET=(0.01, 1, 0.01), UT=(0.01, 1, 0.01), K2=(0.01, 1, 0.01),continuous_update=False)

In [18]:
w

interactive(children=(FloatSlider(value=1.0, description='k1', max=10.0, min=0.1, step=0.01), FloatSlider(valu…

In [19]:
def show_sol(k1 = 1., k2 = 1., r3 = 1., ET = 0.5, r1=1, UT=0.1, r2=0.075, J3 = 0.3, K2 = 0.3, k4 = 0.2, J4 = 0.5):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        Y = x[0]
        VP = G(r3 * Y, k4, J3, J4) # equation (3)
        UP = x[1]
        dYdT = k1*VP -k2*UP*Y
        dUPdT = r1*ET*((UT-UP)/(k1+(UT-UP))-r2*Y*(UP/(K2+UP)))
        return [dYdT, dUPdT]



    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(R,X)
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w2 = interactive(show_sol, k1=(0.1, 10, 0.01), K1=(0.1, 10, 0.01),r3=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), r1=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), r2=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), ET=(0.01, 10, 0.01), UT=(0.01, 10, 0.01), K2=(0.01, 10, 0.01),continuous_update=False)

In [20]:
w2

interactive(children=(FloatSlider(value=1.0, description='k1', max=10.0, min=0.1, step=0.01), FloatSlider(valu…

In [21]:
def show_sol(k1 = 1., k2 = 1., r3 = 1., ET = 0.5, r1=1, UT=0.1, r2=0.075, J3 = 0.3, K2 = 0.3, k4 = 0.2, J4 = 0.5):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        Y = x[0]
        VP = G(r3 * Y, k4, J3, J4) # equation (3)
        UP = x[1]
        dYdT = k1*VP -k2*UP*Y
        dUPdT = r1*ET*((UT-UP)/(k1+(UT-UP))-r2*Y*(UP/(K2+UP)))
        return [dYdT, dUPdT]



    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(R,X)
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w3 = interactive(show_sol, k1=(0.1, 10, 0.01), K1=(0.1, 10, 0.01),r3=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), r1=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), r2=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), ET=(0.01, 10, 0.01), UT=(0.01, 10, 0.01), K2=(0.01, 10, 0.01),continuous_update=False)

In [22]:
w3

interactive(children=(FloatSlider(value=1.0, description='k1', max=10.0, min=0.1, step=0.01), FloatSlider(valu…

In [35]:
def show_sol(k1 = 2.87, k2 = 3.65, r3 = 4.94, ET = 2.34, r1=2.07, UT=2.62, r2=1.52, J3 =0.30, K2 =1.90, k4 = 1.35, J4 =1.46):
    def CAMP_PDEP(x, t):
        def G(u, v, J, K): # equation (4)
            numerator = 2 * u * K # numerator of the fraction
            den_1 = v - u + v * J + u * K # first part of the denominator
            den_2 = -4 * (v - u) * u * K # second part of the denominator
            radicand = den_1**2 + den_2 # expression under the square root
            denominator = den_1 + np.sqrt(radicand) # compute the entire denominator
            return numerator / denominator # compute and return the fraction
        Y = x[0]
        VP = G(r3 * Y, k4, J3, J4) # equation (3)
        UP = x[1]
        dYdT = k1*VP -k2*UP*Y
        dUPdT = r1*ET*((UT-UP)/(k1+(UT-UP))-r2*Y*(UP/(K2+UP)))
        return [dYdT, dUPdT]



    x0 = [0.1, 0.8] # initial conditions
    solver = RungeKutta4(CAMP_PDEP)
    solver.set_initial_condition(x0)
    time_points = np.linspace(0, 100, 1000)
    x, t = solver.solve(time_points)
    R = x[:,0]; X = x[:,1]
    plt.title("Some title")
    plt.plot(t,X)
    plt.plot(t,R)
    #plt.plot(R,X)
    plt.xlabel('Time')
    plt.ylabel('Concenration')
    plt.legend()
    plt.show()




w3 = interactive(show_sol, k1=(0.1, 10, 0.01), K1=(0.1, 10, 0.01),r3=(0.1, 10, 0.01),  k2=(0.1, 10, 0.01), r1=(0.01, 10, 0.01), k3=(0.1, 10, 0.01), k4=(0.1, 10, 0.01), r2=(0.01, 10, 0.01), J4=(0.1, 10, 0.01), J3=(0.1, 10, 0.01), ET=(0.01, 10, 0.01), UT=(0.01, 10, 0.01), K2=(0.01, 10, 0.01),continuous_update=False)

In [36]:
w3

interactive(children=(FloatSlider(value=2.87, description='k1', max=10.0, min=0.1, step=0.01), FloatSlider(val…